**https://github.com/aladdinpersson/Machine-Learning-Collection/tree/master/ML/Pytorch/Basics/custom_dataset**

In [ ]:
!pip install tensorflow tensorboard tf-keras-vis grad-cam
import os
import pandas as pd
import torch
from torch.utils.data import Dataset
from skimage import io
from PIL import Image
import matplotlib.pyplot as plt

class CheXpertDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.annotations = self.annotations.fillna(0) ######### USE HERE BECAUSE CHEXPERT IS MULT-Class

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir, self.annotations.iloc[index, 0])
        image = io.imread(img_path)
        image = Image.fromarray(image).convert('RGB')  # Convert the NumPy array to a PIL Image
        y_label = torch.tensor(self.annotations.iloc[index, 5:19].values.astype(float)) #######  add colums for chexpert, 14 lables ## 5:19].values.astype(float))

        if self.transform:
            image = self.transform(image)

        return image, y_label

%load_ext tensorboard

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 57.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 5.5 MB/s eta 0:00:00
  Created wheel for grad-cam: filename=grad_cam-1.5.4-py3-none-any.whl size=39587 sha256=75537115dc94c1afd0eb4fd8f71c297c6f8372d884c8762f0bfa1d059aff0ac4
  Stored in directory: /root/.cache/pip/wheels/50/b0/82/1f97b5348c7fe9f0ce0ba18497202cafa5dec4562bd5292680
Successfully built grad-cam


In [ ]:
from google.colab import output

# JavaScript code to prevent timeout
code = """
function KeepClicking(){
    console.log("Clicking to prevent timeout");
    document.querySelector("colab-toolbar-button").click();
}
setInterval(KeepClicking, 60000);
"""

output.eval_js(code)

32

In [ ]:
#%tensorboard --logdir logs/resnet

# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import models
from pathlib import Path
from PIL import Image
#import tmm

from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter(log_dir='./logs/resnet')

# Set device (use GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device,"||",torch.device)

# Load Emphysema_Dataset  (host domain)
transform = transforms.Compose([
  transforms.Resize((224, 224)),  # Resize images to the size expected by ResNet50 originally 1024 x1024
  transforms.ToTensor(),
  transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),  # Normalize based on ImageNet stats
])


trainData = "/kaggle/input/chexpert/CheXpert-v1.0-small/train.csv"
testData = "/kaggle/input/chexpert/CheXpert-v1.0-small/valid.csv"
trainImgDir = "/kaggle/input/chexpert"
testImgDir = "/kaggle/input/chexpert"

train_dataset = CheXpertDataset(csv_file=trainData, root_dir=trainImgDir, transform=transform)
test_dataset = CheXpertDataset(csv_file=testData, root_dir=testImgDir, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=900, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=900, shuffle=False)

cuda || <class 'torch.device'>


In [ ]:
def modelResnet():

    model = models.resnet18(pretrained=False)


    #model = timm.create_model('xception', pretrained=True)
    #model.eval()

    model.fc = nn.Linear(model.fc.in_features, 14)  # CheXpert has 14 classes
    model.load_state_dict(torch.load('/content/drive/My Drive/trained_model.pth'))  # Load the saved state

    model = model.to(device)

    # Define loss function and optimizer
    criterion = nn.BCEWithLogitsLoss() ######## https://discuss.pytorch.org/t/loss-function-crossentropyloss-vs-bcewithlogitsloss/16089
    optimizer = optim.Adam(model.parameters(), lr=0.001)


    return model, criterion, optimizer

In [ ]:
from google.colab import drive
import torch

# Mount Google Drive
drive.mount('/content/drive')

def train_model(model, train_loader, criterion, optimizer, num_epochs):
    model.train()
    print("Training for", num_epochs, "epochs")

    for epoch in range(num_epochs):
        print(f'Epoch {epoch + 1}/{num_epochs}')
        print('-' * 10)
        running_loss = 0.0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # Log training loss to TensorBoard
        epoch_loss = running_loss / len(train_loader)
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}')
        writer.add_scalar('Training Loss', epoch_loss, epoch + 1)

        # Save the model state every 10 epochs and overwrite old model
        if (epoch + 1) % 2 == 0:
            model_path = '/content/drive/My Drive/trained_model.pth'
            torch.save(model.state_dict(), model_path)
            print(f"Model saved to Google Drive after epoch {epoch + 1}")

    # At the end of training, save the model
    final_model_path = '/content/drive/My Drive/trained_model_final.pth'
    torch.save(model.state_dict(), final_model_path)
    print("Final model saved to Google Drive.")

    writer.close()

# Example usage:
# model, criterion, optimizer = modelResnet()
# train_model(model, train_loader, criterion, optimizer, num_epochs=50)

Mounted at /content/drive


In [ ]:
# Train and evaluate the model
#model = Image.fromarray(model)
model, criterion, optimizer = modelResnet()

epochs = 0



/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
<ipython-input-5-8db6afb046b8>:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions

In [ ]:
train_model(model, train_loader, criterion, optimizer, num_epochs=int(epochs))

Training for 0 epochs
Final model saved to Google Drive.


In [ ]:
f1, accuracy = test_model(model, test_loader, criterion)
print(f"F1 Score: {f1:.4f}")
print(f"Accuracy: {accuracy:.4f}")

F1 Score: 0.0590
Accuracy of the model on the test images: 11.11%
F1 Score: 0.0590
Accuracy: 11.1111


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
writer.close()
# Save the trained model

In [ ]:
torch.save(model.state_dict(), '/content/drive/trained_model.pth')
torch.save(model.state_dict(), 'trained_model.pth')


In [ ]:
%tensorboard --logdir logs/resnet